In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
import requests

%config Completer.use_jedi = False

#### Step 1: Read and Prepare Data

In [2]:
data = pd.read_csv('cleveland.csv')

my_filter = data['diagnosis'] == 0
data.loc[my_filter, 'heart_disease'] = 0
data.loc[~my_filter, 'heart_disease'] = 1
data = data.drop(columns = ['diagnosis'])
data.columns = [col.replace(' ', '_') for col in data.columns]

for col in data.columns:
    data[col] = pd.to_numeric(data[col], errors = 'coerce')
    data[col] = data[col].fillna(data[col].median())

#### Step 2: Make Requests to Deployed HTTP Endpoints

In [3]:
neural_url = 'http://127.0.0.1:5000/neural'
adaboost_url = 'http://127.0.0.1:5000/adaboost'

In [4]:
features = [
    'age', 
    'sex', 
    'chest_pain', 
    'blood_pressure', 
    'serum_cholestoral',
    'fasting_blood_sugar', 
    'electrocardiographic', 
    'max_heart_rate',
    'induced_angina', 
    'ST_depression', 
    'slope', 
    'vessels', 
    'thal',
]

In [5]:
index = 100
data_dict = data.loc[index, features].to_dict()
data_dict

{'age': 45.0,
 'sex': 1.0,
 'chest_pain': 4.0,
 'blood_pressure': 115.0,
 'serum_cholestoral': 260.0,
 'fasting_blood_sugar': 0.0,
 'electrocardiographic': 2.0,
 'max_heart_rate': 185.0,
 'induced_angina': 0.0,
 'ST_depression': 0.0,
 'slope': 1.0,
 'vessels': 0.0,
 'thal': 3.0}

In [6]:
response = requests.post(neural_url, json = data_dict)
nn_prediction = response.json()

response = requests.post(adaboost_url, json = data_dict)
ab_prediction = response.json()

print(f'{index}: NN -> {nn_prediction}, Ada -> {ab_prediction}')   

100: NN -> 0, Ada -> 0
